In [12]:
import numpy as np

In [16]:
target = np.array([2,3,4,2])
display(target)

pred_choice = np.array([1,3,4,1])
display(pred_choice)

cats = np.unique(target)
display(cats)

cat = cats[0]
a = target == cat
b = pred_choice[a]
display(cat, a, b)

array([2, 3, 4, 2])

array([1, 3, 4, 1])

array([2, 3, 4])

2

array([ True, False, False,  True])

array([1, 1])

In [23]:
a = np.array([0.111111, 0.2222222])

a = ["%.3f" % x for x in a]

"%s" % a

"['0.111', '0.222']"